Usando os conhecimentos adquiridos pra validar dados de uma API que simula de forma simples um gerenciador de bibliotecas/consulta de livros por status.

In [ ]:
!pip install fastapi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.2 MB/s eta 0:00:00


*FastAPI + Pydantic*

In [ ]:
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel, validator
from enum import Enum
import logging

# Criação do modelo de livro
class BookStatus(str, Enum):
    available = "available"
    borrowed = "borrowed"
    reserved = "reserved"


# Modelo de validação para criação de livro
class BookCreate(BaseModel):
    title: str
    author: str
    isbn: str
    status: BookStatus

    @validator("isbn")
    def validate_isbn(cls, v):
        """Valida que o ISBN tenha 13 caracteres"""
        if len(v) != 13:
            raise ValueError("ISBN deve ter 13 caracteres")
        return v

    @validator("status")
    def validate_status(cls, v):
        """Valida que o status seja um valor válido"""
        if v not in BookStatus.__members__:
            raise ValueError(f"Status deve ser um dos seguintes: {', '.join(BookStatus.__members__)}")
        return v

    class Config:
        min_anystr_length = 1  # Define que todos os campos devem ter no mínimo 1 caractere.


# Criando a instância do FastAPI
app = FastAPI()

# Exemplo de good data
good_data = {
    "title": "Harry Potter",
    "author": "J.K. Rowling",
    "isbn": "978-3-16-148410-0",
    "status": "available"
}

# Exemplo de bad data
bad_data = {
    "title": "The Catcher in the Rye",
    "author": "J.D. Salinger",
    "isbn": "12345",  # ISBN inválido
    "status": "lost"  # Status inválido
}

# Endpoint para mostrar a validação de "good data"
@app.get("/good_data/")
async def get_good_data():
    try:
        book = BookCreate(**good_data)
        return {"message": "Livro válido", "book": book.dict()}
    except Exception as e:
        return {"error": str(e)}

# Endpoint para mostrar a validação de "bad data"
@app.get("/bad_data/")
async def get_bad_data():
    try:
        book = BookCreate(**bad_data)
        return {"message": "Livro válido", "book": book.dict()}
    except Exception as e:
        return {"error": str(e)}


<ipython-input-2-5db653b716ca>:20: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("isbn")
<ipython-input-2-5db653b716ca>:27: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("status")
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'min_anystr_length' has been renamed to 'str_min_length'
  warnings.warn(message, UserWarning)
